# Project - deep learning modeling and optimization

In this project you'll be required to implement an architecture of a network, train it on dataset while comparing different optimizers and eventually optimize it using TensorRT.

## Implement and train the model
Hereby shown the architecture of a well known classifier VGG-19:

|Layer Type|	Feature Map|	Size	|Kernel Size|	Stride	|Activation|
| :-: | :-: | :-: | :-: | :-: | :-: |
|Image|	1	|224×224|	–|	–|	–|
|Convolution|	64|	224×224|	3×3|	1|	ReLU|
|Convolution|	64|	224×224|	3×3|	1|	ReLU|
|Max Pooling|	64|	112×112|	2×2|	2|	–|
|Convolution|	128|	112×112|	3×3|	1|	ReLU|
|Convolution|	128|	112×112|	3×3|	1|	ReLU|
|Max Pooling|	128|	56×56|	2×2|	2|	–|
|Convolution|	256|	56×56|	3×3|	1|	ReLU|
|Convolution|	256|	56×56|	3×3|	1|	ReLU|
|Convolution|	256|	56×56|	3×3|	1|	ReLU|
|Convolution|	256|	56×56|	3×3|	1|	ReLU|
|Max Pooling|	256|	28×28|	2×2|	2|	–|
|Convolution|	512|	28×28|	3×3|	1|	ReLU|
|Convolution|	512|	28×28|	3×3|	1|	ReLU|
|Convolution|	512|	28×28|	3×3|	1|	ReLU|
|Convolution|	512|	28×28|	3×3|	1|	ReLU|
|Max Pooling|	512|	14×14|	2×2|	2|	–|
|Convolution|	512|	14×14|	3×3|	1|	ReLU|
|Convolution|	512|	14×14|	3×3|	1|	ReLU|
|Convolution|	512|	14×14|	3×3|	1|	ReLU|
|Convolution|	512|	14×14|	3×3|	1|	ReLU|
|Max Pooling|	512|	7×7|	2×2|	2|	–|
|Fully Connected|	–|	4096|	–|	–|	ReLU|
|Fully Connected|	–|	4096|	–|	–|	ReLU|
|Fully Connected|	–|	1000|	–|	–|	Softmax|

Please implement this network architecture in tensorflow and train it using the imagenet_v2 dataset (you can use tensorflow integrated dataset for imagenet_v2 - https://www.tensorflow.org/datasets/catalog/imagenet_v2)

While training please explore usage of two different optimizers, compare the both and state your selection and reason.

Use crossentropy for loss function (categorical crossentropy if working with categorical labels).

Choose the proper metrics to evaluate model performance and perform model evaluation.


In [1]:
# import necessary layers

from tensorflow.keras.layers import Input, Conv2D
from tensorflow.keras.layers import MaxPool2D, Flatten, Dense
from tensorflow.keras import Model

In [2]:
# input  
input = Input(shape =(224,224,3))


In [3]:
# Block 1
x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv1')(input)
x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv2')(x)
x = MaxPool2D((2, 2), strides=(2, 2), name='block1_pool')(x)

# Block 2
x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv1')(x)
x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv2')(x)
x = MaxPool2D((2, 2), strides=(2, 2), name='block2_pool')(x)

# Block 3
x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv1')(x)
x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv2')(x)
x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv3')(x)
x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv4')(x)
x = MaxPool2D((2, 2), strides=(2, 2), name='block3_pool')(x)

# Block 4
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv1')(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv2')(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv3')(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv4')(x)
x = MaxPool2D((2, 2), strides=(2, 2), name='block4_pool')(x)

# Block 5
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv1')(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv2')(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv3')(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv4')(x)
x = MaxPool2D((2, 2), strides=(2, 2), name='block5_pool')(x)


In [4]:
# Fully connected layers  
x = Flatten()(x) 
x = Dense(units = 4096, activation ='relu')(x) 
x = Dense(units = 4096, activation ='relu')(x) 
output = Dense(units = 1000, activation ='softmax')(x)

In [5]:
# creating the model

model = Model (inputs=input, outputs =output)
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [42]:
import tensorflow as tf
import tensorflow_datasets as tfds

# tfds works in both Eager and Graph modes
# tf.enable_eager_execution()

# See available datasets
# print(tfds.list_builders())

# Construct a tf.data.Dataset
(train,test),info = tfds.load(name="cifar100", split=['train','test'],as_supervised=True,with_info=True,data_dir='./data_jacobd')

In [23]:
import cv2


In [44]:
def resize_img(image,label):
    return tf.image.resize(image,(224,224))
train = train.map(resize_img)
test = test.map(resize_img)

In [45]:
train

<MapDataset shapes: (224, 224, 3), types: tf.float32>

In [ ]:
trainX = trainX.reshape((trainX.shape[0], 28, 28, 1))

In [8]:
len(train),len(test)

(50000, 10000)

In [12]:
train

<_OptionsDataset shapes: ((32, 32, 3), ()), types: (tf.uint8, tf.int64)>

In [29]:
opt = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9)
model.compile(optimizer=opt, loss=tf.keras.losses.CategoricalCrossentropy(), metrics=['accuracy'])

In [30]:
model.fit(train, epochs=2)

Epoch 1/2


ValueError: in user code:

    /usr/local/lib/python3.8/dist-packages/tensorflow/python/keras/engine/training.py:855 train_function  *
        return step_function(self, iterator)
    /usr/local/lib/python3.8/dist-packages/tensorflow/python/keras/engine/training.py:845 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /usr/local/lib/python3.8/dist-packages/tensorflow/python/distribute/distribute_lib.py:1285 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /usr/local/lib/python3.8/dist-packages/tensorflow/python/distribute/distribute_lib.py:2833 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /usr/local/lib/python3.8/dist-packages/tensorflow/python/distribute/distribute_lib.py:3608 _call_for_each_replica
        return fn(*args, **kwargs)
    /usr/local/lib/python3.8/dist-packages/tensorflow/python/keras/engine/training.py:838 run_step  **
        outputs = model.t

In [91]:
tk = train.take(1)

for img in tk:
    #print(img)
    print(img[1])
    print(img[0].shape)

tf.Tensor(23, shape=(), dtype=int64)
(32, 32, 3)


In [45]:
train_ds.shape

(50000, 4)

In [39]:
print(x_train.shape,y_train.shape)
print(x_test.shape,y_test.shape)


(50000, 32, 32, 3) (50000,)
(10000, 32, 32, 3) (10000,)


In [ ]:
# Resizing for images

In [11]:
import cv2

## Optimize the model using TensorRT

After training of the model and evaluating it, your goal is to optimize the model for inference on target machine using TensorRT (use TF-TRT in this project).

Try quantizing the model for different percisions using TensorRT quantization features, compare the different percision modes and recommand what you choose.

> Bonus: if you were working on Tesla T4 GPU, what percision mode had you chosen then?